In [7]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers

In [10]:
#   Students_Performance_knn.csv
#   Students_Performance_mv.csv
#   Wine_Quality.csv
df1 = pd.read_csv('data/Students_Performance_knn.csv')
df2 = pd.read_csv('data/Students_Performance_mv.csv')
df3 = pd.read_csv('data/Wine_Quality.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/Students_Performance_knn.csv'

In [9]:
# Drop numerical columns
df2.drop(columns=['math score','reading score','writing score'],inplace=True)

NameError: name 'df2' is not defined

In [154]:
# Find the number of missing values per column.
df2.isnull().sum()

gender                          0
race/ethnicity                 11
parental level of education    21
lunch                          12
test preparation course         4
dtype: int64

In [155]:
df2.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')

In [156]:
# The number of incomplete subsets
m = 5
# The number of attributes
s = df2.columns.size    # 8

In [157]:
def ExtractCompleteTuples(df):
    # getting the rows without null values
    CT = df.dropna()
    # print(CT.shape) #(959, 8)
    # print(CT.min())
    CTTransposed = CT.T
    return CT
    
x=ExtractCompleteTuples(df2)
y=np.array(x)
y=y.transpose()
y[1][835]

'group C'

In [158]:
def ExtractInCompleteTuples(df):
    # getting only the rows with null values
    ICT = df[df.isnull().any(axis=1)]
    # print(ICT.shape) #(41, 8)
    return ICT.values

ExtractInCompleteTuples(df2)

array([['female', nan, 'some college', 'free/reduced', 'completed'],
       ['female', nan, "associate's degree", 'free/reduced', 'none'],
       ['male', 'group D', nan, 'standard', 'completed'],
       ['male', 'group A', nan, nan, 'completed'],
       ['female', nan, nan, 'standard', 'none'],
       ['female', 'group E', 'some college', nan, 'completed'],
       ['female', nan, "associate's degree", 'free/reduced', 'none'],
       ['male', 'group C', "bachelor's degree", nan, 'none'],
       ['male', 'group A', 'high school', nan, 'completed'],
       ['female', 'group B', "bachelor's degree", nan, 'none'],
       ['male', 'group C', 'high school', nan, 'none'],
       ['male', 'group E', "associate's degree", nan, 'completed'],
       ['female', 'group C', 'some high school', nan, 'none'],
       ['male', 'group E', nan, nan, 'completed'],
       ['male', 'group B', nan, 'standard', 'completed'],
       ['female', 'group C', nan, 'free/reduced', 'completed'],
       ['male', 'group

In [159]:
def min(val):
    m = val[0]
    for item in val:
        if item < m:
            m = item
    return m

def max(val):
    m = val[0]
    for item in val:
        if item > m:
            m = item
    return m

In [160]:
sna = [0]*5
print(sna)

[0, 0, 0, 0, 0]


In [161]:
from math import log,e

# Entropy weight method (EWM)
def ComputeAttributeWeights(CT):
    # CTTransposed = CT.T (assume that CT is already transposed)
    n = CT.shape[0]
    s = CT.shape[1]

    # 1- Normalizing data(just numerocal cols)
#     y = [s][n]
#     sna = [0] * s # will create a list of size s full of zeros
#     for i in range(s):
#         for j in range(n):
#             y[i][j] = (CT[i][j] - min(CT[i])) / (max(CT[i] - min(CT[i])))
#             sna[i] += y[i][j]


    # 2-1 Calculating the entropy of each numerical attribute   
#     p = [s][n]
#     container = [0] * s
    
#     for i in range(s):
#         for j in range(n):
#             p[i][j] = y[i][j] / sna[i]
#             container[i] += p[i][j] * log(p[i][j])

#     E = [0] * s
#     print(E)
#     for i in range(s):
#         for j in range(n):
            
#             E[i] = -log(container[i] / n)
    
    
    # 2-2 Calculating the entropy of each categorical attribute 
    def entropy(labels, base=None):
        vc = pd.Series(labels).value_counts(normalize=True, sort=False)
        base = e if base is None else base
        return -(vc * np.log(vc)/np.log(base)).sum()

    E=[]
    for column in CT:
        E.append(entropy(CT[column], base=None))
    
    # 3- Determining the weight of each attribute
    w = [0] * s
    k = s
    sum = 0
    for i in range(k):
        sum += E[i]
    for i in range(s):
        w[i] = (1 - E[i]) / (k - sum)

    return w

In [162]:
def SortInCompleteTuples(ICT, r):
    # Convert list to npArray
    r = np.array(r)

    # Arg sort
    argSort = np.argsort(r) # it sorts r, and returns corresponding indexes
    
    # Create new empty npArray for sorted ICT
    sortedICT = np.copy(ICT)
    for index in range(argSort.size):
        sortedICT[index] = ICT[argSort[index]]
    
    return sortedICT

# SortInCompleteTuples(ICT, [4, 8, 2])

In [163]:
def Partition(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [164]:
def GenerateTuplePartition(sortedICT, m):
    T=[]
    T=Partition(sortedICT, m)
    return T

In [165]:
def GenerateTuplePartitions(ICT, CT, m, s):

    W = ComputeAttributeWeights(CT)
    
    # STEP 1
    # Calculate tuple intergrity rate, according to DEFINITION 5(example)
    inCompleteRowsCount = ICT.shape[0]      # gives us the number of ICT rows
    r = [1] * inCompleteRowsCount                      
    for i in range(inCompleteRowsCount):
        for j in range(s):
            if pd.isnull(ICT[i][j]):               # if ICT[i][j] item, not exists then go through
                r[i] = r[i] - W[j]    
                
    # at this part we're sure that we have intergrity rate's list

    # STEP 2
    # sort ICT's tuples according to their intergrity rate
    sortedICT = SortInCompleteTuples(ICT, r)
    
    # STEP 3
    tuplePartitions = GenerateTuplePartition(sortedICT, m)
    return tuplePartitions # a queue of subsets

In [166]:
# Defining a function which calculates euclidean distance between two data points(numerical)
def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

In [167]:
def weighted_hamming(data):
    categories_dist = []
    
    for category in data:
        X = pd.get_dummies(data[category])
        X_mean = X * X.mean()
        X_dot = X_mean.dot(X.transpose())
        X_np = np.asarray(X_dot.replace(0,1,inplace=False))
        categories_dist.append(X_np)
        
    categories_dist = np.array(categories_dist)
    distances = hmean(categories_dist, axis=0)
    return distances

In [168]:
def distance_matrix(data, numeric_distance = "euclidean", categorical_distance = "jaccard"):
    number_of_variables = data.shape[1]
    number_of_observations = data.shape[0]

    # Get the type of each attribute (Numeric or categorical)
    is_numeric = [all(isinstance(n, numbers.Number) for n in data.iloc[:, i]) for i, x in enumerate(data)]
    is_all_categorical = sum(is_numeric) == 0

    # Replace missing values with mode for categorical columns
    for x in data:
        data[x].fillna(data[x].mode()[0], inplace=True)

    # "Dummifies" categorical variables in place
    if not (categorical_distance == 'hamming' or categorical_distance == 'weighted-hamming'):
        data = pd.get_dummies(data)
    elif categorical_distance == 'hamming':
        data = pd.DataFrame([pd.factorize(data[x])[0] for x in data]).transpose()

    if is_all_categorical:
        if categorical_distance == "weighted-hamming":
            result_matrix = weighted_hamming(data)
        else:
            result_matrix = cdist(data, data, metric=categorical_distance)

    # Fill the diagonal with NaN values
    np.fill_diagonal(result_matrix, np.nan)
    return pd.DataFrame(result_matrix)

In [169]:
def knn_impute(target, attributes, k_neighbors, aggregation_method="mode", numeric_distance="euclidean",
               categorical_distance="jaccard", missing_neighbors_threshold = 0.5):
    
    number_observations = len(target)
    is_target_numeric = all(isinstance(n, numbers.Number) for n in target)
    numberOfSamples=len(target[0])
   
    # Make sure the data are in the right format
    target = pd.DataFrame(target)
    attributes = pd.DataFrame(attributes)
    
    # Get the distance matrix and check whether no error was triggered when computing it
    distances = distance_matrix(attributes.transpose(), numeric_distance, categorical_distance)

    # Get the closest points and compute the correct aggregation method
    for j in range(numberOfSamples):
        for i, value in enumerate(target.iloc[:, j]):
            if pd.isnull(value):
                order = distances.iloc[i,:].values.argsort()[:k_neighbors]
                closest_to_target = attributes.iloc[i, order]
                target.iloc[i,j]=stats.mode(closest_to_target)[0][0]
    
    return target

In [170]:
def KNNImputation(train_set, test_set):
    knn_impute(test_set.transpose(),train_set.transpose(), k_neighbors=10)

In [203]:
def Merge(a, b):
    a=np.array(a)
    b=np.array(b)
    return np.concatenate((a, b), axis=0)

In [223]:
# Begin
CT= ExtractCompleteTuples(df2)   # this is dataframe
ICT = ExtractInCompleteTuples(df2) # this is npArray

m=4 # test 
T   = GenerateTuplePartitions(ICT, CT, m, s)

CTS=[[0]] * (m)
CTS[0]=np.array(CT.copy())

for i in range(1, m):
    KNNImputation(CTS[i-1], T[i])
    CTS[i] =  Merge(CTS[i-1], T[i])
    
print(CTS)     
# Do cross validation
# for i in range(1, m-1):


[array([['female', 'group B', "bachelor's degree", 'standard', 'none'],
       ['female', 'group C', 'some college', 'standard', 'completed'],
       ['female', 'group B', "master's degree", 'standard', 'none'],
       ...,
       ['female', 'group C', 'high school', 'free/reduced', 'completed'],
       ['female', 'group D', 'some college', 'standard', 'completed'],
       ['female', 'group D', 'some college', 'free/reduced', 'none']],
      dtype=object), array([['female', 'group B', "bachelor's degree", 'standard', 'none'],
       ['female', 'group C', 'some college', 'standard', 'completed'],
       ['female', 'group B', "master's degree", 'standard', 'none'],
       ...,
       ['male', 'group D', "master's degree", 'standard', 'completed'],
       ['male', 'group C', "master's degree", 'standard', 'completed'],
       ['female', 'group C', 'some college', 'free/reduced', 'none']],
      dtype=object), array([['female', 'group B', "bachelor's degree", 'standard', 'none'],
       ['